# OgreInterface Tutorial

This notebook demonstrates how to use OgreInterface for creating and optimizing epitaxial interfaces.

## Initialize OgreInterface
Run once at the start of your session.

In [ ]:
from OgreInterface.workflows.OgreSupport import initialize_Ogre, lattice_matching, visualize_supercells, \
    surface_matching, mlip_relaxation

initialize_Ogre(check_install=False)  # True to auto-install / update Ogre. It might take a while

## Lattice Matching
Import the CIF files of the two materials, and explore the relative orientations resulting in promising lattice matches.

In [ ]:
CIF_folder = "./Reference CIFs/"  # Folder where the CIF files are located
material_A = "CsPbBr3_cubic"       # CIF of material A (do not add .cif!)
material_B = "Pb4S3Br2"            # CIF of material B (do not add .cif!)
max_hkl_A = 2                      # max h,k,l for material A
max_hkl_B = 1                      # max h,k,l for material B
max_strain = 0.1                   # max strain (1 = 100%)
max_area = "default"               # max supercell area (put a number to override OgreInterface default)
sort_by_area_first = True          # True: sort interface matches prioritizing area; False: prioritize strain.
image_dpi = 300
image_scale = 1

# 'results' contains the lattice matching result in pandas.DataFrame format
results = lattice_matching(CIF_folder, material_A, material_B, max_hkl_A, max_hkl_B, max_strain, max_area, image_dpi, image_scale,
                           sort_by_area_first=sort_by_area_first)

## Interface Generation
Use this block to visualize and select the supercell that you are going to optimize in the next block.

In [ ]:
hkl_A = [1, 0, 0]      # hkl choice for material A
hkl_B = [0, 1, 0]      # hkl choice for material B
mode = "view_best"     # Visualization mode for supercells. Options = "view_best", "view_all", or integer number to see the first N options
strain_fraction = 0.5  # Strain partition coefficient: 0 = strain fully in material B (film, e.g., in epitaxial systems), 1 = strain fully in material A (substrate), 0.5 = strain equally shared (e.g., in nanocrystals).
is_molecule = False    # True if the system is molecular, False if it is inorganic (e.g., oxide, perovskite, etc.). Affect the generation of the interfaces.

visualize_supercells(CIF_folder, material_A, material_B, hkl_A, hkl_B, mode, image_dpi, max_strain, max_area,
                     strain_fraction, image_scale, is_molecule=is_molecule, sort_by_area_first=sort_by_area_first)

## Surface Matching and Ranking
Use this block to optimize the interface models for one of the supercells from the previous block.

In [ ]:
supercell_choice = 0         # Supercell index as seen in the Interface Generation block. Use 0 to select the best interface according to Ogre
minimum_slab_thickness = 18  # Minimum slab thickness (in Å) for substrate and film during interface generation and surface matching.
label = " - classical"       # Label added at the end of the folder name, useful for notes. Use "" for no label.

surface_matching(
    CIF_folder, material_A, material_B, hkl_A, hkl_B, label, max_strain, max_area, strain_fraction, supercell_choice,
    minimum_slab_thickness=minimum_slab_thickness,
    is_molecule=is_molecule,                  # True if the system is molecular, False if it is inorganic (e.g., oxide, perovskite, etc.). Affect the generation of the interfaces.
    opt_method = "PSO",                       # Optimization method for surface matching. Currently, only "PSO" (Particle Swarm Optimization) and "BO" (Bayesian Optimization) are supported, with "PSO" as the default.
    n_particles_PSO=30,                       # Number of particles in the PSO algorithm. Higher number = more accurate but slower.
    Z_bounds=None,                            # List of two floats specifying the bounds for interfacial distance optimization.
                                              # If None, OgreInterface will use default values based on covalent radii.
    substrate_layer_grouping_tolerance=None,  # Tolerance for grouping substrate layers. If None, OgreInterface will use default values based on maximum interatomic distances.
    film_layer_grouping_tolerance=None,       # Tolerance for grouping film layers. If None, OgreInterface will use default values based on maximum interatomic distances.
    filter_on_charge=True,                    # If True, OgreInterface will filter out terminations based on residual surface charge.
    sort_by_area_first=sort_by_area_first,    # If True, sort interface matches prioritizing area. If False, prioritize strain.
    create_dft_inputs=False,                  # If True, create input files for DFT calculations for interface ranking validation.
    create_dft_inputs_xyz_scan=False,         # If True, create input files for DFT calculations for potential energy surface (PES) and binding curve.
    mlip=None
)